# Generating Data Columns using CellUtils

This notebook shows how to use the `get_data_cols` function to generate data columns and save a pyhton pickle object for use in training, scoring, and plotting hits

## Files needed

A cellprofile sqlite measurement file

In [ ]:
import sqlite3

import padnas as pd
import numpy as np

from cellutils.utils import get_data_cols

fname = "path/to/my/cellprofiler/output.db"

con = sqltie3.connect(fname)

Run this cell to get table names

- Most measurement tables are `Per_Object`

In [ ]:
sql_query = """SELECT name FROM sqlite_master  
  WHERE type='table';"""

cursor = con.cursor()
cursor.execute(sql_query)
print(cursor.fetchall())

With the table name use the cell bellow to read it as a dataframe

In [ ]:
tablename= "Per_Object"
query = f"SELECT * FROM {tablename}"

chunks = []
for chunk in pd.read_sql_query(query, con, chunksize=int(1e6)):
    print("Read chunk of size: ", len(chunk))
    chunks.append(chunk)
df = pd.concat(chunks)
del chunks
df.head()

## Generating Data columns

For this example we need to remove extra meta columns of `current_temp` and `gas_price` since they do not add to the measurements. They will be added to a list of strings called `extra` and we will save the data columns as pickle file called `data_cols` in the `data` directory

**Important Note:**

We print out the data columns after if there are any extra columns that need to be removed add them as a new string the the extra list

In [ ]:
extra=['current_temp', 'gas_price']

data_cols = get_data_cols(df, extra=extra, save=True, fname='data/data_cols')

for c in data_cols:
    print(c)

To illustrate the need for removing metadata columns run this cell

In [ ]:
print("With all columns: ", df.shape, "\nWith only Data columns: ", df[data_cols].shape)